In [8]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

In [9]:
df1 = pd.read_excel('Telco_customer_churn_demographics.xlsx')
df2 = pd.read_excel('Telco_customer_churn_location.xlsx')
df3 = pd.read_excel('Telco_customer_churn_services.xlsx')
df4 = pd.read_excel('Telco_customer_churn_status.xlsx')
df5 = pd.read_excel('Telco_customer_churn_population.xlsx')

df1.columns = df1.columns.str.replace(' ', '_')
df2.columns = df2.columns.str.replace(' ', '_')
df3.columns = df3.columns.str.replace(' ', '_')
df4.columns = df4.columns.str.replace(' ', '_')
df5.columns = df5.columns.str.replace(' ', '_')

In [10]:
pd.set_option('display.max_columns', 200)

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df3.head()

In [ ]:
df4.head()

In [ ]:
df5.head()

In [ ]:
print('df1 has the shape of: ', df1.shape)
print('df2 has the shape of: ', df2.shape)
print('df3 has the shape of: ', df3.shape)
print('df4 has the shape of: ', df4.shape)
print('df5 has the shape of: ', df5.shape)

In [ ]:
merged_df = pd.merge(df1, df2, on='Customer_ID', how='outer')
print(merged_df.shape)

In [ ]:
merged_df1 = pd.merge(merged_df, df3, on='Customer_ID', how='outer')
print(merged_df1.shape)

In [ ]:
merged_df2 = pd.merge(merged_df1, df4, on='Customer_ID', how='outer')
print(merged_df2.shape)

In [ ]:
merged_df3 = pd.merge(merged_df2, df5, on='Zip_Code', how='outer')
print(merged_df3.shape)

In [ ]:
merged_df3.head()

In [ ]:
nan_check_df = merged_df3.isnull().any()
nan_check_df.shape

In [ ]:
total_nan_count = merged_df3.isnull().sum()
total_nan_count

In [ ]:
unique_zip_df2 = df2['Zip_Code'].nunique()
print("Unique values of Zip_Code in df2:", unique_zip_df2)

In [ ]:
unique_zip_df5 = df5['Zip_Code'].nunique()
print("Unique values of Zip_Code in df5:", unique_zip_df5)

In [ ]:
num_unique_values1 = merged_df2['Zip_Code'].nunique()
print("Number of unique values:", num_unique_values1)

In [ ]:
num_unique_values2 = merged_df3['Zip_Code'].nunique()
print("Number of unique values:", num_unique_values2)

In [ ]:
print('Difference of unique values: ', num_unique_values2 - num_unique_values1)

In [29]:
merged_df3.dropna(subset=['Customer_ID'], inplace=True)

In [ ]:
nan_count = merged_df3.isna().sum()
print(nan_count)

In [ ]:
merged_df3.columns

In [32]:
col_to_remove = ['Customer_ID', 'Count_x', 'Under_30', 'Number_of_Dependents','Location_ID',
                 'Count_y', 'Country', 'State', 'City', 'Zip_Code', 'Lat_Long', 'Latitude',
                 'Longitude', 'Service_ID', 'Count_x', 'Quarter_x', 'Number_of_Referrals',
                 'Total_Refunds', 'Status_ID', 'Count_y', 'Quarter_y', 'Satisfaction_Score',
                 'Customer_Status', 'Churn_Value', 'Churn_Score', 'CLTV', 'Churn_Category',
                 'Churn_Reason', 'ID', 'Population']

mer_fin = merged_df3.drop(columns=col_to_remove)

In [33]:
mer_fin.to_csv('merged.csv', index=False)

In [ ]:
data = pd.read_csv('merged.csv')
data.shape

In [ ]:
!pip install -U ydata.profiling

In [36]:
from ydata_profiling import ProfileReport
telco_profile = ProfileReport(data, title = 'Telco Profiling Report')

In [ ]:
telco_profile

In [ ]:
telco_profile.to_file('Report.html')

In [ ]:
html = telco_profile.to_html()
html[:200]

In [ ]:
# Use eval function to convert to a Dictionary
json_report = telco_profile.to_json()
json_report[:200]

In [41]:
from ydata_profiling import ProfileReport
telco_profile = ProfileReport(data, title = 'Telco Profiling Report', minimal = True)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data, train_size = 0.8, stratify=data['Churn_Label'])
train_df.shape, test_df.shape

In [ ]:
from ydata_profiling import ProfileReport, compare
train_report = ProfileReport(train_df, title='Train Datafrme')
test_report = ProfileReport(test_df, title='Test Datafrme')
comparison_report = compare([train_report, test_report])

In [ ]:
comparison_report

In [ ]:
comparison_report.to_file('Comparison.html')

In [ ]:
# Check the range of numbers and measures of central tendency for numerical columns

# Extract numerical columns
numerical_columns = data.select_dtypes(include=['float64']).columns

# Display range and measures of central tendency for each numerical column
numerical_stats = data[numerical_columns].describe()

numerical_stats

In [ ]:
# Check for special codes or non-nullable fields
special_codes = (data == 999).sum()

# Check for duplicate records
duplicates = data.duplicated().sum()

print(special_codes[special_codes > 0]),
print(duplicates)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Identify outliers using the Tukey method (Boxplot)
plt.figure(figsize=(15, 10))
sns.boxplot(data=data[numerical_columns])
plt.xticks(rotation=90)
plt.title('Boxplot of Numerical Columns')
plt.show()

In [ ]:
# Check for skewness in the data
skewness = data[numerical_columns].skew()

skewness

In [ ]:
import numpy as np

data['Avg_Monthly_GB_Download_log'] = np.log10(data['Avg_Monthly_GB_Download'] + 1)
print("Avg_Monthly_GB_Download_log:\n", data['Avg_Monthly_GB_Download_log'].skew())

data['Total_Extra_Data_Charges'] = np.log10(data['Total_Extra_Data_Charges'] + 1)
print("\nTotal_Extra_Data_Charges:\n", data['Total_Extra_Data_Charges'].skew())

data['Total_Long_Distance_Charges'] = np.log10(data['Total_Long_Distance_Charges'] + 1)
print("\nTotal_Long_Distance_Charges:\n", data['Total_Long_Distance_Charges'].skew())

In [ ]:
data['Total_Extra_Data_Charges'] = np.log10(data['Total_Extra_Data_Charges'] + 1)
print("\nTotal_Extra_Data_Charges:\n", data['Total_Extra_Data_Charges'].skew())

In [ ]:
data['Total_Extra_Data_Charges'] = np.log2(data['Total_Extra_Data_Charges'] + 1)
print("\nTotal_Extra_Data_Charges:\n", data['Total_Extra_Data_Charges'].skew())

In [ ]:
# Visualize histograms for the numerical columns
data[numerical_columns].hist(figsize=(15, 12), bins=50)
plt.suptitle('Histograms of Numerical Columns')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# Visualize bar charts for the categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns
fig, axes = plt.subplots(nrows=len(categorical_columns), figsize=(10, 60))

for i, col in enumerate(categorical_columns):
    sns.countplot(data=data, x=col, ax=axes[i])
    axes[i].set_title(f'Bar Chart of {col}')
    axes[i].set_ylabel('Count')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze the relationship between the target variable (Churn_Label) and other variables

# Visualize the distribution of the target variable
plt.figure(figsize=(8, 6))
sns.countplot(data=data, x='Churn_Label')
plt.title('Distribution of Churn_Label')
plt.ylabel('Count')
plt.show()

In [ ]:
# Visualize the relationship between Churn_Label and categorical variables
categorical_columns = data.select_dtypes(include=['object']).columns
fig, axes = plt.subplots(nrows=len(categorical_columns) - 1, figsize=(10, 50))  # Exclude Churn_Label itself

for i, col in enumerate(categorical_columns):
    if col != 'Churn_Label':  # Exclude Churn_Label itself
        sns.countplot(data=data, x=col, hue='Churn_Label', ax=axes[i])
        axes[i].set_title(f'Relationship between {col} and Churn_Label')
        axes[i].set_ylabel('Count')
        axes[i].tick_params(axis='x', rotation=45)

In [ ]:
# Visualize the relationship between Churn_Label and numerical variables using boxplots
fig, axes = plt.subplots(nrows=len(numerical_columns), figsize=(10, 40))

for i, col in enumerate(numerical_columns):
    sns.boxplot(data=data, x='Churn_Label', y=col, ax=axes[i])
    axes[i].set_title(f'Relationship between {col} and Churn_Label')
    axes[i].set_ylabel(col)

plt.tight_layout()
plt.show()

In [ ]:
# Compute the correlation matrix
correlation_matrix = data.corr().round(2)

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1,  fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Identify pairs of variables with high correlation (greater than 0.7)
high_corr_pairs = {}
for col in correlation_matrix.columns:
    for index in correlation_matrix.index:
        if (0.7 < correlation_matrix.loc[index, col] < 1.0) and (col, index) not in high_corr_pairs and (index, col) not in high_corr_pairs:
            high_corr_pairs[(index, col)] = correlation_matrix.loc[index, col]

high_corr_pairs

In [ ]:
from scipy.stats import chi2_contingency

# Analyze the relationship between the target variable and categorical variables using chi-square tests
chi2_results = {}
for col in categorical_columns:
    if col != 'Churn_Label':  # Exclude the target variable itself
        contingency_table = pd.crosstab(data[col], data['Churn_Label'])
        chi2, p, _, _ = chi2_contingency(contingency_table)
        chi2_results[col] = p

# Convert the results into a DataFrame for better visualization
chi2_df = pd.DataFrame(list(chi2_results.items()), columns=['Variable', 'P-Value'])
chi2_df.sort_values(by='P-Value', inplace=True)
chi2_df

In [ ]:
from scipy.stats import f_oneway

# Analyze the relationship between numeric and categorical variables using ANOVA
anova_results = {}
for cat_col in categorical_columns:
    if cat_col != 'Churn_Label':  # Exclude the target variable itself
        for num_col in numerical_columns:
            groups = [data[num_col][data[cat_col] == category] for category in data[cat_col].unique()]
            f_stat, p_value = f_oneway(*groups)
            anova_results[(cat_col, num_col)] = p_value

# Convert the results into a DataFrame for better visualization
anova_df = pd.DataFrame(list(anova_results.items()), columns=['(Categorical, Numerical)', 'P-Value'])
anova_df.sort_values(by='P-Value', inplace=True)
anova_df.head(10)

In [ ]:
data.columns

In [ ]:
data['Churn_Label'].value_counts()

In [ ]:
print((data['Churn_Label'].value_counts()/len(mer_fin['Churn_Label']))*100)

In [ ]:
data['Churn_Label'].value_counts().plot(kind='bar', figsize=(8, 6))
plt.xlabel("COUNT", labelpad=16)
plt.ylabel("CHURN", labelpad=16)
plt.title("Count of CHURN", y=1.00)

In [ ]:
# Visualization of key features in relation to Churn_Label

features = ['Gender', 'Senior_Citizen', 'Married']

plt.figure(figsize=(15, 10))
for i, feature in enumerate(features, 1):
    plt.subplot(2, 2, i)
    sns.countplot(x=feature, hue='Churn_Label', data=data, palette='pastel')
    plt.title(f'Distribution of {feature} by Churn_Label')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.legend(title='Churn_Label')

plt.tight_layout()
plt.show()

In [ ]:
# List of categorical variables
categorical_features = data.select_dtypes(include=['object']).columns.tolist()
categorical_features.remove('Churn_Label')  # Remove target variable

# Plot bar charts for churn for each categorical variable
plt.figure(figsize=(20, 30))
for i, feature in enumerate(categorical_features, 1):
    plt.subplot(6, 4, i)
    sns.countplot(x=feature, hue='Churn_Label', data=data, palette='pastel')
    plt.title(f'Distribution of {feature} by Churn_Label')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.legend(title='Churn_Label')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Reformat Data - Ordinal
# Assign numbers to the 'Contract' column to represent their order
contract_mapping = {'Month-to-Month': 1, 'One Year': 2, 'Two Year': 3}
data['Contract_Ordinal'] = data['Contract'].map(contract_mapping)
data[['Contract', 'Contract_Ordinal']].head(10)

In [ ]:
# 1. Remove Duplicate Rows
initial_rows = data.shape[0]
data.drop_duplicates(inplace=True)
removed_duplicates = initial_rows - data.shape[0]

# 2. Remove Constant Columns
constant_columns = [col for col in data.columns if data[col].nunique() == 1]
data.drop(columns=constant_columns, inplace=True)

removed_duplicates, constant_columns

In [ ]:
# Columns to remove based on high correlation
columns_to_remove = ['Referred_a_Friend', 'Total_Charges', 'Internet_Type',
                     'Avg_Monthly_GB_Download_log', 'Streaming_Music',
                     'Contract_Ordinal', 'Total_Long_Distance_Charges']

data.drop(columns=columns_to_remove, inplace=True)

# Display the first few rows after removing the columns
data.head()
print(data.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Identify categorical columns
categorical_cols = data.select_dtypes(include=['object']).columns

# Apply label encoding
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Display the first few rows after encoding
data.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Identify numerical columns
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
numerical_cols = numerical_cols.drop('Churn_Label')  # Exclude target variable

# Apply standard scaling
scaler = StandardScaler()
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

# Display the first few rows after scaling
data.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the correlation matrix
correlation_matrix = data[numerical_cols].corr().round(2)  # Round values to 2 decimal places

# Identify pairs of features with high correlation
high_corr_pairs = {}
for col in correlation_matrix.columns:
    for idx in correlation_matrix.index:
        if col != idx and abs(correlation_matrix.loc[idx, col]) > 0.7:
            sorted_pair = tuple(sorted([col, idx]))
            if sorted_pair not in high_corr_pairs:
                high_corr_pairs[sorted_pair] = correlation_matrix.loc[idx, col]

# Plot the correlation heatmap
plt.figure(figsize=(25, 15))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".2f")  # Use fmt to format annotations
plt.title('Feature Correlation Heatmap')
plt.show()

In [ ]:
data.columns

In [ ]:
# Decision Tree with original target distribution and labelled

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Splitting the data into training and testing sets (80% train, 20% test)
X = data.drop('Churn_Label', axis=1)
y = data['Churn_Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training a simple decision tree model
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# Evaluation

dtclf_ori_accuracy = accuracy_score(y_test, y_pred)
dtclf_ori_classification_rep = classification_report(y_test, y_pred)
dtclf_ori_confusion_mat = confusion_matrix(y_test, y_pred)

print('The accuracy score for this model is: ', dtclf_ori_accuracy),
print('\nThe classification report for this model: \n', dtclf_ori_classification_rep),
print('\nThe confusion matrix for this model: \n', dtclf_ori_confusion_mat)

In [ ]:
# Decision Tree with original target distribution and on dummies

# Identifying categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Encoding categorical columns
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Splitting the encoded data into training and testing sets
X_train_encoded, X_test_encoded, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Training the decision tree model on the encoded data
clf_encoded = DecisionTreeClassifier(random_state=42)
clf_encoded.fit(X_train_encoded, y_train)

# Predictions
y_pred_encoded = clf_encoded.predict(X_test_encoded)

# Evaluation
dtclf_ori_accuracy_encoded = accuracy_score(y_test, y_pred_encoded)
dtclf_ori_classification_rep_encoded = classification_report(y_test, y_pred_encoded)
dtclf_ori_confusion_mat_encoded = confusion_matrix(y_test, y_pred_encoded)

print('The accuracy for this model is: ', dtclf_ori_accuracy_encoded),
print('\nThe classification report for this model: \n', dtclf_ori_classification_rep_encoded),
print('\nThe confusion matrix for this model: \n', dtclf_ori_confusion_mat_encoded)

In [ ]:
from sklearn import tree
# Initialize the Decision Tree Classifier with a depth of 7
dt_classifier_depth7 = DecisionTreeClassifier(max_depth=7, random_state=42)

# Fit the classifier to the training data
dt_classifier_depth7.fit(X_train, y_train)

# Plot the decision tree
fig, ax = plt.subplots(figsize=(20, 15))
tree.plot_tree(dt_classifier_depth7, feature_names=X_train.columns, class_names=['No Churn', 'Churn'], filled=True, rounded=True, fontsize=10)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
rf_model = RandomForestClassifier()

# Train the model
rf_model.fit(X_train_encoded, y_train)

# Extracting feature importances
feature_importances = rf_model.feature_importances_
features = X_train_encoded.columns

# Creating a DataFrame for feature importances
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plotting the top 7 features based on importance
plt.figure(figsize=(12, 8))
importance_df.head(7).plot(kind='bar', x='Feature', y='Importance', legend=False, color='skyblue')
plt.title('Top 7 Features based on Importance')
plt.ylabel('Importance')
plt.xlabel('Feature')
plt.xticks(rotation=45)
plt.show()
print('Top 7 Features based on Importance in Descending order are: \n', importance_df.head(7))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Predict on test data
y_pred = rf_model.predict(X_test_encoded)

#Random Forest Classifier with encoded data and original target distribution
ranfor_ori_accuracy = accuracy_score(y_test, y_pred)
ranfor_ori_precision = precision_score(y_test, y_pred)
ranfor_ori_recall = recall_score(y_test, y_pred)
ranfor_ori_f1 = f1_score(y_test, y_pred)
ranfor_ori_roc_auc = roc_auc_score(y_test, y_pred)
ranfor_ori_classification_rep = classification_report(y_test, y_pred)
ranfor_ori_confusion_mat = confusion_matrix(y_test, y_pred)

print('The accuracy for this model is: ', ranfor_ori_accuracy),
print('\nThe precision score for this model: \n', ranfor_ori_precision),
print('\nThe recall score for this model: \n', ranfor_ori_recall),
print('\nThe F1 score for this model: \n', ranfor_ori_f1),
print('\nThe roc_auc score for this model: \n', ranfor_ori_roc_auc),
print('\nThe classification report for this model: \n', ranfor_ori_classification_rep),
print('\nThe confusion matrix for this model: \n', ranfor_ori_confusion_mat)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Splitting the data into training and testing sets (80% train, 20% test)
X_train_simple, X_test_simple, y_train_simple, y_test_simple = train_test_split(X_train_encoded, y_train, test_size=0.2, random_state=42)

# Training a Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_simple, y_train_simple)

# Predicting on the test set
y_pred_simple = log_reg.predict(X_test_simple)

# Logistic Regression with encoded data and original target distribution
logreg_ori_accuracy = accuracy_score(y_test_simple, y_pred_simple)
logreg_ori_precision = precision_score(y_test_simple, y_pred_simple)
logreg_ori_recall = recall_score(y_test_simple, y_pred_simple)
logreg_ori_f1 = f1_score(y_test_simple, y_pred_simple)
logreg_ori_roc_auc = roc_auc_score(y_test_simple, y_pred_simple)
logreg_ori_classification_rep = classification_report(y_test_simple, y_pred_simple)
logreg_ori_conf_matrix = confusion_matrix(y_test_simple, y_pred_simple)

print('The accuracy for this model is: ', logreg_ori_accuracy),
print('\nThe precision score for this model: \n', logreg_ori_precision),
print('\nThe recall score for this model: \n', logreg_ori_recall),
print('\nThe F1 score for this model: \n', logreg_ori_f1),
print('\nThe roc_auc score for this model: \n', logreg_ori_roc_auc),
print('\nThe classification report for this model: \n', logreg_ori_classification_rep),
print('\nThe confusion matrix for this model: \n', logreg_ori_conf_matrix)

In [ ]:
# Checking the correlation of each feature with the target variable
correlation_with_target = X_train_encoded.join(y_train).corr()['Churn_Label'].sort_values(ascending=False)
correlation_with_target

In [ ]:
# Initialize the Decision Tree Classifier with a depth of 5
dt_classifier_depth5 = DecisionTreeClassifier(max_depth=5, random_state=42)

# Fit the classifier to the training data
dt_classifier_depth5.fit(X_train, y_train)

# Plot the decision tree
fig, ax = plt.subplots(figsize=(20, 15))
tree.plot_tree(dt_classifier_depth5, feature_names=X_train.columns, class_names=['No Churn', 'Churn'], filled=True, rounded=True, fontsize=10)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

# Predict on the test dataset
y_pred_depth5 = dt_classifier_depth5.predict(X_test)

# Calculate the accuracy
dtclf_ori_accuracy_depth5 = accuracy_score(y_test, y_pred_depth5)
dtclf_ori_precision_depth5 = precision_score(y_test, y_pred_depth5)
dtclf_ori_recall_depth5 = recall_score(y_test, y_pred_depth5)
dtclf_ori_f1_depth5 = f1_score(y_test, y_pred_depth5)
dtclf_ori_roc_auc_depth5 = roc_auc_score(y_test, y_pred_depth5)
dtclf_ori_classification_rep_depth5 = classification_report(y_test, y_pred_depth5)
dtclf_ori_conf_matrix_depth5 = confusion_matrix(y_test, y_pred_depth5)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(dtclf_ori_conf_matrix_depth5, annot=True, fmt='g', cmap='Blues', cbar=False,
            xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title(f'Confusion Matrix (Accuracy: {dtclf_ori_accuracy_depth5:.2f})')
plt.show()

print('The accuracy for this model is: ', dtclf_ori_accuracy_depth5),
print('\nThe precision score for this model: \n', dtclf_ori_precision_depth5),
print('\nThe recall score for this model: \n', dtclf_ori_recall_depth5),
print('\nThe F1 score for this model: \n', dtclf_ori_f1_depth5),
print('\nThe roc_auc score for this model: \n', dtclf_ori_roc_auc_depth5),
print('\nThe classification report for this model: \n', dtclf_ori_classification_rep_depth5),
print('\nThe confusion matrix for this model: \n', dtclf_ori_conf_matrix_depth5)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
logreg_model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model on the training data
logreg_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Predict churn labels for the test set
y_pred = logreg_model.predict(X_test)

# Logistic Regression with encoded data and original target distribution
logreg_ori_accuracy = accuracy_score(y_test, y_pred)
logreg_ori_precision = precision_score(y_test, y_pred)
logreg_ori_recall = recall_score(y_test, y_pred)
logreg_ori_f1 = f1_score(y_test, y_pred)
logreg_ori_roc_auc = roc_auc_score(y_test, y_pred)
logreg_ori_classification_rep = classification_report(y_test, y_pred)
logreg_ori_conf_matrix = confusion_matrix(y_test, y_pred)


# Plot confusion matrix
plt.figure(figsize=(8,6))
sns.heatmap(logreg_ori_conf_matrix, annot=True, fmt='g', cmap='Blues', cbar=False,
            xticklabels=['Not Churned', 'Churned'],
            yticklabels=['Not Churned', 'Churned'])
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

print('The accuracy for this model is: ', logreg_ori_accuracy),
print('\nThe precision score for this model: \n', logreg_ori_precision),
print('\nThe recall score for this model: \n', logreg_ori_recall),
print('\nThe F1 score for this model: \n', logreg_ori_f1),
print('\nThe roc_auc score for this model: \n', logreg_ori_roc_auc),
print('\nThe classification report for this model: \n', logreg_ori_classification_rep),
print('\nThe confusion matrix for this model: \n', logreg_ori_conf_matrix)

In [ ]:
from sklearn.metrics import classification_report
import pandas as pd

# Assuming y_test and y_pred are already defined
report_dict = classification_report(y_test, y_pred, output_dict=True)

# Convert the dictionary to a DataFrame
report_df = pd.DataFrame(report_dict).transpose()

print(report_df)


In [ ]:
# Encoding Categorical Variables using One-Hot Encoding
data_encoded = pd.get_dummies(data, drop_first=True)

# Display the first few rows of the encoded dataset
data_encoded.head()

In [ ]:
!pip install -q imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE

# Splitting features and target variable
X = data_encoded.drop('Churn_Label', axis=1)
y = data_encoded['Churn_Label']

# Applying SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Checking the distribution of the target variable after SMOTE
y_resampled.value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the resampled features
X_resampled_scaled = scaler.fit_transform(X_resampled)

# Display the first few rows of the scaled features
X_resampled_scaled[:5]

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_resampled_scaled, y_resampled, test_size=0.2, random_state=42)

# Display the shape of the training and testing sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# Initialize and train the Logistic Regression model
logreg = LogisticRegression(random_state=42, max_iter=1000)
logreg.fit(X_train, y_train)

# Predict on the test set
y_pred_logreg = logreg.predict(X_test)

# Logistic Regression after SMOTE
logreg_smo_accuracy = accuracy_score(y_test, y_pred_logreg)
logreg_smo_precision = precision_score(y_test, y_pred_logreg)
logreg_smo_recall = recall_score(y_test, y_pred_logreg)
logreg_smo_f1 = f1_score(y_test, y_pred_logreg)
logreg_smo_roc_auc = roc_auc_score(y_test, y_pred_logreg)
logreg_smo_classification_report = classification_report(y_test, y_pred_logreg)
logreg_smo_conf_matrix = confusion_matrix(y_test, y_pred_logreg)

print('The accuracy for this model is: ', logreg_smo_accuracy),
print('\nThe precision score for this model: \n', logreg_smo_precision),
print('\nThe recall score for this model: \n', logreg_smo_recall),
print('\nThe F1 score for this model: \n', logreg_smo_f1),
print('\nThe roc_auc score for this model: \n', logreg_smo_roc_auc),
print('\nThe classification report for this model: \n', logreg_smo_classification_report),
print('\nThe confusion matrix for this model: \n', logreg_smo_conf_matrix)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Initialize and train the Decision Trees model
dtree = DecisionTreeClassifier(random_state=42)
dtree.fit(X_train, y_train)

# Predict on the test set
y_pred_dtree = dtree.predict(X_test)

# Decision tree after SMOTE
dtclf_smo_accuracy = accuracy_score(y_test, y_pred_dtree)
dtclf_smo_precision = precision_score(y_test, y_pred_dtree)
dtclf_smo_recall = recall_score(y_test, y_pred_dtree)
dtclf_smo_f1 = f1_score(y_test, y_pred_dtree)
dtclf_smo_roc_auc = roc_auc_score(y_test, y_pred_dtree)
dtclf_smo_classification_report = classification_report(y_test, y_pred_dtree)
dtclf_smo_conf_matrix = confusion_matrix(y_test, y_pred_dtree)

print('The accuracy for this model is: ', dtclf_smo_accuracy),
print('\nThe precision score for this model: \n', dtclf_smo_precision),
print('\nThe recall score for this model: \n', dtclf_smo_recall),
print('\nThe F1 score for this model: \n', dtclf_smo_f1),
print('\nThe roc_auc score for this model: \n', dtclf_smo_roc_auc),
print('\nThe classification report for this model: \n', dtclf_smo_classification_report),
print('\nThe confusion matrix for this model: \n', dtclf_smo_conf_matrix)

In [ ]:
!pip install -q xgboost

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the Random Forest model
rf = RandomForestClassifier(random_state=42, n_estimators=100)
rf.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = rf.predict(X_test)

# Random Forest Classifier after SMOTE
ranfor_smo_accuracy = accuracy_score(y_test, y_pred_rf)
ranfor_smo_precision = precision_score(y_test, y_pred_rf)
ranfor_smo_recall = recall_score(y_test, y_pred_rf)
ranfor_smo_f1 = f1_score(y_test, y_pred_rf)
ranfor_smo_roc_auc = roc_auc_score(y_test, y_pred_rf)
ranfor_smo_classification_report = classification_report(y_test, y_pred_rf)
ranfor_smo_conf_matrix = confusion_matrix(y_test, y_pred_rf)


print('The accuracy for this model is: ', ranfor_smo_accuracy),
print('\nThe precision score for this model: \n', ranfor_smo_precision),
print('\nThe recall score for this model: \n', ranfor_smo_recall),
print('\nThe F1 score for this model: \n', ranfor_smo_f1),
print('\nThe roc_auc score for this model: \n', ranfor_smo_roc_auc),
print('\nThe classification report for this model: \n', ranfor_smo_classification_report),
print('\nThe confusion matrix for this model: \n', ranfor_smo_conf_matrix)

In [ ]:
import xgboost as xgb

# Initialize and train the XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42, n_estimators=100)
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred_xgb = xgb_model.predict(X_test)

# XGBClassifier after SMOTE
xgb_smo_accuracy = accuracy_score(y_test, y_pred_xgb)
xgb_smo_precision = precision_score(y_test, y_pred_xgb)
xgb_smo_recall = recall_score(y_test, y_pred_xgb)
xgb_smo_f1 = f1_score(y_test, y_pred_xgb)
xgb_smo_roc_auc = roc_auc_score(y_test, y_pred_xgb)
xgb_smo_classification_report = classification_report(y_test, y_pred_xgb)
xgb_smo_conf_matrix = confusion_matrix(y_test, y_pred_xgb)

print('The accuracy for this model is: ', xgb_smo_accuracy),
print('\nThe precision score for this model: \n', xgb_smo_precision),
print('\nThe recall score for this model: \n', xgb_smo_recall),
print('\nThe F1 score for this model: \n', xgb_smo_f1),
print('\nThe roc_auc score for this model: \n', xgb_smo_roc_auc),
print('\nThe classification report for this model: \n', xgb_smo_classification_report),
print('\nThe confusion matrix for this model: \n', xgb_smo_conf_matrix)

In [ ]:
from sklearn.svm import SVC

# Initialize and train the SVM model
svm_model = SVC(random_state=42, probability=True)
svm_model.fit(X_train, y_train)

# Predict on the test set
y_pred_svm = svm_model.predict(X_test)

# SVC after SMOTE
svm_smo_accuracy = accuracy_score(y_test, y_pred_svm)
svm_smo_precision = precision_score(y_test, y_pred_svm)
svm_smo_recall = recall_score(y_test, y_pred_svm)
svm_smo_f1 = f1_score(y_test, y_pred_svm)
svm_smo_roc_auc = roc_auc_score(y_test, y_pred_svm)
svm_smo_classification_report = classification_report(y_test, y_pred_svm)
svm_smo_conf_matrix = confusion_matrix(y_test, y_pred_svm)


print('The accuracy for this model is: ', svm_smo_accuracy),
print('\nThe precision score for this model: \n', svm_smo_precision),
print('\nThe recall score for this model: \n', svm_smo_recall),
print('\nThe F1 score for this model: \n', svm_smo_f1),
print('\nThe roc_auc score for this model: \n', svm_smo_roc_auc),
print('\nThe classification report for this model: \n', svm_smo_classification_report),
print('\nThe confusion matrix for this model: \n', svm_smo_conf_matrix)

In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.metrics import Precision, Recall, AUC

# Define the Neural Network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy',
                       Precision(name='precision'),
                       Recall(name='recall'),
                       AUC(name='auc_roc', curve='ROC'),  # AUC for ROC curve
                       AUC(name='auc_pr', curve='PR')    # AUC for Precision-Recall curve
                       ]
            )

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=1)

In [ ]:
import pandas as pd

# Convert the history.history dictionary to a pandas DataFrame
history_df = pd.DataFrame(history.history)

# Optionally, add an epoch column
history_df['epoch'] = history.epoch

# Display the DataFrame
print(history_df)

In [ ]:
# Save to CSV
history_df.to_csv('training_history.csv', index=False)

# Save to Excel
history_df.to_excel('training_history.xlsx', index=False)

In [ ]:
import matplotlib.pyplot as plt

# Setting up the size of the plots
plt.figure(figsize=(20, 10))

# Plotting Accuracy
plt.subplot(2, 3, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Plotting Loss
plt.subplot(2, 3, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plotting Precision
plt.subplot(2, 3, 3)
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('Model Precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Plotting Recall
plt.subplot(2, 3, 4)
plt.plot(history.history['recall'])
plt.plot(history.history['val_recall'])
plt.title('Model Recall')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Plotting AUC for ROC
plt.subplot(2, 3, 5)
plt.plot(history.history['auc_roc'])
plt.plot(history.history['val_auc_roc'])
plt.title('Model AUC for ROC')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Plotting AUC for PR
plt.subplot(2, 3, 6)
plt.plot(history.history['auc_pr'])
plt.plot(history.history['val_auc_pr'])
plt.title('Model AUC for PR')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Adjusting the layout
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [50, 100, 200],
    'subsample': [0.5, 0.7, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.7, 0.9, 1.0]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print('The best parameters are : \n', best_params)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize the model
knn = KNeighborsClassifier(n_neighbors=5)  # Using 5 neighbors for KNN

# Train the model
knn.fit(X_train, y_train)

# Prediction
knn_preds = knn.predict(X_test)

# Evaluation

# KNN after SMOTE
knn_smo_accuracy = accuracy_score(y_test, knn_preds)
knn_smo_precision = precision_score(y_test, knn_preds)
knn_smo_recall = recall_score(y_test, knn_preds)
knn_smo_f1 = f1_score(y_test, knn_preds)
knn_smo_roc_auc = roc_auc_score(y_test, knn_preds)
knn_smo_classification_report = classification_report(y_test, knn_preds)
knn_smo_conf_matrix = confusion_matrix(y_test, knn_preds)

print('The knn_accuracy is         : ', knn_smo_accuracy),
print('\nThe precision score for this model: \n', knn_smo_precision),
print('\nThe recall score for this model: \n', knn_smo_recall),
print('\nThe F1 score for this model: \n', knn_smo_f1),
print('\nThe roc_auc score for this model: \n', knn_smo_roc_auc),
print('\nThe classification report for this model: \n', knn_smo_classification_report),
print('\nThe confusion matrix for this model: \n', knn_smo_conf_matrix)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)

naive_bayes_preds = naive_bayes.predict(X_test)

# Naive Bayes after SMOTE
naive_bayes_smo_accuracy = accuracy_score(y_test, naive_bayes_preds)
naive_bayes_smo_precision = precision_score(y_test, naive_bayes_preds)
naive_bayes_smo_recall = recall_score(y_test, naive_bayes_preds)
naive_bayes_smo_f1 = f1_score(y_test, naive_bayes_preds)
naive_bayes_smo_roc_auc = roc_auc_score(y_test, naive_bayes_preds)
naive_bayes_smo_classification_report = classification_report(y_test, naive_bayes_preds)
naive_bayes_smo_conf_matrix = confusion_matrix(y_test, naive_bayes_preds)

print('The naive_bayes_accuracy is : ', naive_bayes_smo_accuracy),
print('\nThe precision score for this model: \n', naive_bayes_smo_precision),
print('\nThe recall score for this model: \n', naive_bayes_smo_recall),
print('\nThe F1 score for this model: \n', naive_bayes_smo_f1),
print('\nThe roc_auc score for this model: \n', naive_bayes_smo_roc_auc),
print('\nThe classification report for this model: \n', naive_bayes_smo_classification_report),
print('\nThe confusion matrix for this model: \n', naive_bayes_smo_conf_matrix)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

ada_boost = AdaBoostClassifier(n_estimators=50)  # Using 50 estimators for AdaBoost
ada_boost.fit(X_train, y_train)

ada_boost_preds = ada_boost.predict(X_test)

# ADA Boost after SMOTE
ada_boost_smo_accuracy = accuracy_score(y_test, ada_boost_preds)
ada_boost_smo_precision = precision_score(y_test, ada_boost_preds)
ada_boost_smo_recall = recall_score(y_test, ada_boost_preds)
ada_boost_smo_f1 = f1_score(y_test, ada_boost_preds)
ada_boost_smo_roc_auc = roc_auc_score(y_test, ada_boost_preds)
ada_boost_smo_classification_report = classification_report(y_test, ada_boost_preds)
ada_boost_smo_conf_matrix = confusion_matrix(y_test, ada_boost_preds)


print('The ada_boost_accuracy is   : ', ada_boost_smo_accuracy)
print('\nThe precision score for this model: \n', ada_boost_smo_precision),
print('\nThe recall score for this model: \n', ada_boost_smo_recall),
print('\nThe F1 score for this model: \n', ada_boost_smo_f1),
print('\nThe roc_auc score for this model: \n', ada_boost_smo_roc_auc),
print('\nThe classification report for this model: \n', ada_boost_smo_classification_report),
print('\nThe confusion matrix for this model: \n', ada_boost_smo_conf_matrix)

In [ ]:
!pip install lime

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import lime
from lime.lime_tabular import LimeTabularExplainer

# Sample data (only a snippet for demonstration)
data = pd.read_csv('merged.csv')
df = pd.DataFrame(data)

# Encode categorical variables
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Split data
X = df.drop('Churn_Label', axis=1)
y = df['Churn_Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest model
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

In [ ]:
# Evaluating the "STATISTICAL IMPORTANCE" of the Variables.
# Define a label encoder
from sklearn.preprocessing import StandardScaler, LabelEncoder
#from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
labelencoder = LabelEncoder()

# Loop through each column in the dataframe and if the column type is object (categorical feature),
# we will use the label encoder to transform it into numeric values
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = labelencoder.fit_transform(data[col])

# Define features X (all columns except 'Churn_Label') and target y ('Churn_Label')
X = data.drop('Churn_Label', axis=1)
y = data['Churn_Label']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)

# Train the model
model.fit(X_train, y_train)

# Get feature importances
importances = model.feature_importances_

# Create a DataFrame for visualization
features = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
})

# Sort the DataFrame by descending importance
features = features.sort_values(by='Importance', ascending=False)

print(features)

In [ ]:
# Create LimeTabularExplainer instance
explainer = LimeTabularExplainer(X_train.values,
                                 mode='classification',
                                 training_labels=y_train,
                                 feature_names=X_train.columns.tolist(),
                                 class_names=label_encoders['Churn_Label'].classes_.tolist(),
                                 discretize_continuous=True)

In [ ]:
len(X_test)

In [ ]:
# Analysis of variables for an individual prediction

# Prompt the user for an observation number
example = int(input('The Test dataset has 2113 observations. Enter the observation number please and press enter: '))

# Ensure the example number is within the range of the test dataset
if 0 <= example < len(X_test):
    exp = explainer.explain_instance(X_test.iloc[example].values, clf.predict_proba)
    exp.show_in_notebook(show_table=True, show_all=False)
else:
    print("Invalid observation number. Please enter a number between 0 and ", len(X_test)-1)

In [ ]:
exp.as_map()

In [ ]:
import pandas as pd

# Create a list of dictionaries with model names and their metrics
models_evaluation = [

    {'Model': 'DeTrClassifier Pre SMOTE', 'Accuracy': dtclf_ori_accuracy,  'Precision': 'N / A',
     'Recall': 'N / A', 'F1': 'N / A', 'ROC_AUC' : 'N / A', 'Confusion Matrix': dtclf_ori_confusion_mat},

    {'Model': 'DeTrCalssifier Pre SMOTE Encoded', 'Accuracy': dtclf_ori_accuracy_encoded, 'Precision': 'N / A',
     'Recall': 'N / A', 'F1': 'N / A', 'ROC_AUC' : 'N / A', 'Confusion Matrix': dtclf_ori_confusion_mat_encoded},

    {'Model': 'RaFoClassifier Pre SMOTE Encoded', 'Accuracy': ranfor_ori_accuracy, 'Precision': ranfor_ori_precision,
     'Recall': ranfor_ori_recall, 'F1': ranfor_ori_f1, 'ROC_AUC' : ranfor_ori_roc_auc, 'Confusion Matrix': ranfor_ori_confusion_mat},

    {'Model': 'LogRegression Pre SMOTE Encoded', 'Accuracy': logreg_ori_accuracy, 'Precision': logreg_ori_precision,
     'Recall': logreg_ori_recall, 'F1': logreg_ori_f1, 'ROC_AUC' : logreg_ori_roc_auc, 'Confusion Matrix': logreg_ori_conf_matrix},

    {'Model': 'DeTrClassifier depth5 Pre SMOTE', 'Accuracy': dtclf_ori_accuracy_depth5, 'Precision': dtclf_ori_precision_depth5,
     'Recall': dtclf_ori_recall_depth5, 'F1': dtclf_ori_f1_depth5, 'ROC_AUC' : dtclf_ori_roc_auc_depth5,
     'Confusion Matrix': dtclf_ori_conf_matrix_depth5},

    {'Model': 'LogRegression Pre SMOTE)', 'Accuracy': logreg_ori_accuracy, 'Precision': logreg_ori_precision,
     'Recall': logreg_ori_recall, 'F1': logreg_ori_f1, 'ROC_AUC' : logreg_ori_roc_auc, 'Confusion Matrix': logreg_ori_conf_matrix},

    {'Model': 'LogRegression after SMOTE', 'Accuracy': logreg_smo_accuracy, 'Precision': logreg_smo_precision,
     'Recall': logreg_smo_recall, 'F1': logreg_smo_f1, 'ROC_AUC' : logreg_smo_roc_auc, 'Confusion Matrix': logreg_smo_conf_matrix},

    {'Model': 'Decision tree after SMOTE', 'Accuracy': dtclf_smo_accuracy, 'Precision': dtclf_smo_precision,
     'Recall': dtclf_smo_recall, 'F1': dtclf_smo_f1, 'ROC_AUC' : dtclf_smo_roc_auc, 'Confusion Matrix': dtclf_smo_conf_matrix},

    {'Model': 'RanForClassifier after SMOTE', 'Accuracy': ranfor_smo_accuracy, 'Precision': ranfor_smo_precision,
     'Recall': ranfor_smo_recall, 'F1': ranfor_smo_f1, 'ROC_AUC' : ranfor_smo_roc_auc, 'Confusion Matrix': ranfor_smo_conf_matrix},

    {'Model': 'XGBClassifier after SMOTE', 'Accuracy': xgb_smo_accuracy, 'Precision': xgb_smo_precision,
     'Recall': xgb_smo_recall, 'F1': xgb_smo_f1, 'ROC_AUC' : xgb_smo_roc_auc, 'Confusion Matrix': xgb_smo_conf_matrix},

     {'Model': 'SVC after SMOTE', 'Accuracy': svm_smo_accuracy, 'Precision': svm_smo_precision,
     'Recall': svm_smo_recall, 'F1': svm_smo_f1, 'ROC_AUC' : svm_smo_roc_auc, 'Confusion Matrix': svm_smo_conf_matrix},

     {'Model': 'KNN after SMOTE', 'Accuracy': knn_smo_accuracy, 'Precision': knn_smo_precision,
     'Recall': knn_smo_recall, 'F1': knn_smo_f1, 'ROC_AUC' : knn_smo_roc_auc, 'Confusion Matrix': knn_smo_conf_matrix},

     {'Model': 'Naive Bayes after SMOTE', 'Accuracy': naive_bayes_smo_accuracy, 'Precision': naive_bayes_smo_precision,
     'Recall': naive_bayes_smo_recall, 'F1': naive_bayes_smo_f1, 'ROC_AUC' : naive_bayes_smo_roc_auc,
     'Confusion Matrix': naive_bayes_smo_conf_matrix},

     {'Model': 'ADA Boost after SMOTE', 'Accuracy': ada_boost_smo_accuracy, 'Precision': ada_boost_smo_precision,
     'Recall': ada_boost_smo_recall, 'F1': ada_boost_smo_f1, 'ROC_AUC' : ada_boost_smo_roc_auc, 'Confusion Matrix': ada_boost_smo_conf_matrix},

]

# Convert to DataFrame
df = pd.DataFrame(models_evaluation)

# Set the display width to a large value
pd.set_option('display.width', 1000)

# Print the DataFrame
print(df)

In [ ]:
df.to_csv('model_scores.csv', index=False)

In [ ]:
data = pd.read_csv('merged.csv')

acceptable_ranges = {}

for column in data.columns:
    if data[column].dtype in ['int64', 'float64']:
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        acceptable_ranges[column] = (Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)
    else:
        acceptable_ranges[column] = data[column].unique()
acceptable_ranges

In [ ]:
caps_and_floors = {}

for column in data.columns:
    if data[column].dtype in ['int64', 'float64']:
        cap = data[column].quantile(0.99)
        floor = data[column].quantile(0.01)
        caps_and_floors[column] = (floor, cap)
caps_and_floors

In [ ]:
from scipy.stats import ks_2samp, chisquare

def check_drift(original_data, new_data):
    drift_columns = []
    for column in original_data.columns:
        if original_data[column].dtype in ['int64', 'float64']:
            statistic, p_value = ks_2samp(original_data[column], new_data[column])
            if p_value < 0.05:
                drift_columns.append(column)
        else:
            observed = new_data[column].value_counts().reindex(original_data[column].unique()).fillna(0)
            expected = original_data[column].value_counts().reindex(original_data[column].unique()).fillna(0)
            statistic, p_value = chisquare(observed, expected)
            if p_value < 0.05:
                drift_columns.append(column)
    return drift_columns